# Cấu hình môi trường

In [ ]:
%%bash
git clone https://github.com/hmthanh/GCAT.git

Cloning into 'GCAT'...
Checking out files: 100% (75/75), done.


In [ ]:
!rm -rf /content/data/
!yes | cp --i /content/GCAT/* /content/

cp: overwrite '/content/create_batch.py'? cp: overwrite '/content/create_dataset_files.py'? cp: -r not specified; omitting directory '/content/GCAT/data'
cp: overwrite '/content/layers.py'? cp: overwrite '/content/main.py'? cp: overwrite '/content/models.py'? cp: -r not specified; omitting directory '/content/GCAT/output'
cp: overwrite '/content/prepare.sh'? cp: overwrite '/content/preprocess.py'? cp: overwrite '/content/pytorch35.yml'? cp: overwrite '/content/README.md'? cp: overwrite '/content/utils.py'? 

In [ ]:
%%bash
mv /content/GCAT/* /content/
rm -r /content/GCAT/
rm -r /content/sample_data/

mv: cannot move '/content/GCAT/output' to '/content/output': Directory not empty
rm: cannot remove '/content/sample_data/': No such file or directory


Load Google Drive if you want to save model on Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [1]:
import torch

from models import SpKBGATModified, SpKBGATConvOnly
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from copy import deepcopy

from preprocess import read_entity_from_id, read_relation_from_id, init_embeddings, build_data
from create_batch import Corpus
from utils import save_model
from create_dataset_files import getID

import random
import argparse
import os
import sys
import logging
import time

# import torch_xla
# import torch_xla.core.xla_model as xm
# TPU = xm.xla_device()

Size of train_marked set set  86835
Size of VALID_marked set set  3034
Size of test_marked set set  3134


In [6]:
#@title Cấu hình cài đặt quá trình huấn luyện { run: "auto" }
#@markdown Thư mục hiện tại
root_folder = "./" #@param {type: "string"}

#@markdown Siêu tham số

dataset = "WN18" #@param ["WN18RR", "WN18", "FB15k", "FB15k-237"] {allow-input: false}
Use_Google_Drive_Save_Data = True #@param {type: "boolean"}
drive_folder = "drive/My Drive/" #@param {type: "string"}

CUDA = True #@param {type: "boolean"}
epochs_gat = 3600 #@param {type: "number"}
epochs_conv = 200 #@param {type: "number"}
weight_decay_gat = float(5e-6) #@param {type:"raw"}
weight_decay_conv = float(1e-5) #@param {type:"raw"}
pretrained_emb = True #@param {type: "boolean"}
embedding_size = 50 #@param {type: "slider", min: 20, max: 200}
lr = float(1e-3) #@param {type:"raw"}
get_2hop = True #@param {type: "boolean"}
use_2hop = True #@param {type: "boolean"}
partial_2hop = False #@param {type: "boolean"}
output_folder = "output/"

#@markdown --- Tham số cho mô hình GAT
batch_size_gat = 86835 #@param {type: "number"}
valid_invalid_ratio_gat = 2 #@param {type: "slider", min: 1, max: 10}
drop_GAT = 0.3  #@param {type:"raw"}
alpha = 0.2  #@param {type:"raw"}
entity_out_dim = [100, 200]  #@param {type:"raw"}
nheads_GAT = [2, 2]  #@param {type:"raw"}
margin = 5 #@param {type: "slider", min: 1, max: 10}

#@markdown --- Tham số cho mô hình ConvKB
batch_size_conv = 128  #@param {type: "slider", min: 16, max: 512, step:16}
alpha_conv = 0.2  #@param {type:"raw"}
valid_invalid_ratio_conv = 40
out_channels = 500 #@param {type: "slider", min: 100, max: 1000, step:100}
drop_conv = 0.0  #@param {type:"raw"}

class Args:
    dataset = dataset
    data = "{root_folder}data/{dataset}/".format(root_folder=root_folder, dataset = dataset)
    drive = root_folder + drive_folder
    output_folder = "{root_folder}output/{dataset}/".format(root_folder=root_folder, dataset=dataset)
    if (Use_Google_Drive_Save_Data):
        output_folder = drive

    epochs_gat = epochs_gat
    epochs_conv = epochs_conv
    weight_decay_gat = weight_decay_gat
    weight_decay_conv = weight_decay_conv
    pretrained_emb = pretrained_emb
    embedding_size = embedding_size
    lr = lr
    get_2hop = get_2hop
    use_2hop = use_2hop
    partial_2hop = partial_2hop
    output_folder = output_folder
    

    # Tham số cho mô hình GAT
    batch_size_gat = batch_size_gat
    valid_invalid_ratio_gat = valid_invalid_ratio_gat
    drop_GAT = drop_GAT
    alpha = alpha
    entity_out_dim = entity_out_dim
    nheads_GAT = nheads_GAT
    margin = margin

    # Tham số cho mô hình ConvKB
    batch_size_conv = batch_size_conv
    alpha_conv = alpha_conv
    valid_invalid_ratio_conv = valid_invalid_ratio_conv
    out_channels = out_channels
    drop_conv = drop_conv

args = Args()

# 1. Preprocessing

In [ ]:
def getID(folder=''):
    lstEnts = {}
    lstRels = {}
    with open(folder + 'train.txt') as f, open(folder + 'train_marked.txt', 'w') as f2:
        count = 0
        for line in f:
            line = line.strip().split()
            line = [i.strip() for i in line]
            # print(line[0], line[1], line[2])
            if line[0] not in lstEnts:
                lstEnts[line[0]] = len(lstEnts)
            if line[1] not in lstRels:
                lstRels[line[1]] = len(lstRels)
            if line[2] not in lstEnts:
                lstEnts[line[2]] = len(lstEnts)
            count += 1
            f2.write(str(line[0]) + '\t' + str(line[1]) +
                     '\t' + str(line[2]) + '\n')
        print("Size of train_marked set set ", count)

    with open(folder + 'valid.txt') as f, open(folder + 'valid_marked.txt', 'w') as f2:
        count = 0
        for line in f:
            line = line.strip().split()
            line = [i.strip() for i in line]
            # print(line[0], line[1], line[2])
            if line[0] not in lstEnts:
                lstEnts[line[0]] = len(lstEnts)
            if line[1] not in lstRels:
                lstRels[line[1]] = len(lstRels)
            if line[2] not in lstEnts:
                lstEnts[line[2]] = len(lstEnts)
            count += 1
            f2.write(str(line[0]) + '\t' + str(line[1]) +
                     '\t' + str(line[2]) + '\n')
        print("Size of VALID_marked set set ", count)

    with open(folder + 'test.txt') as f, open(folder + 'test_marked.txt', 'w') as f2:
        count = 0
        for line in f:
            line = line.strip().split()
            line = [i.strip() for i in line]
            # print(line[0], line[1], line[2])
            if line[0] not in lstEnts:
                lstEnts[line[0]] = len(lstEnts)
            if line[1] not in lstRels:
                lstRels[line[1]] = len(lstRels)
            if line[2] not in lstEnts:
                lstEnts[line[2]] = len(lstEnts)
            count += 1
            f2.write(str(line[0]) + '\t' + str(line[1]) +
                     '\t' + str(line[2]) + '\n')
        print("Size of test_marked set set ", count)

    wri = open(folder + 'entity2id.txt', 'w')
    for entity in lstEnts:
        wri.write(entity + '\t' + str(lstEnts[entity]))
        wri.write('\n')
    wri.close()

    wri = open(folder + 'relation2id.txt', 'w')
    for entity in lstRels:
        wri.write(entity + '\t' + str(lstRels[entity]))
        wri.write('\n')
    wri.close()

getID(folder="/content/data/{dataset}/".format(dataset=args.dataset))

Size of train_marked set set  141442
Size of VALID_marked set set  5000
Size of test_marked set set  5000


In [ ]:
!pip install ampligraph

# Load graph

In [13]:
data_folder = "{root_folder}data/{dataset}/".format(root_folder=root_folder, dataset=args.dataset)
train_data, validation_data, test_data, entity2id, relation2id, headTailSelector, unique_entities_train = build_data(path=
        data_folder, is_unweigted=False, directed=True)

KeyError: '_hyponym'

In [12]:
entity2id = read_entity_from_id(data_folder + 'entity2id.txt')
relation2id = read_relation_from_id(data_folder + 'relation2id.txt')
len(relation2id)

11

In [ ]:
#(train_triples, train_adjacency_mat) = train_data
(test_triples, test_adjacency_mat) = test_data
print("test_triples", np.array(test_triples).shape)
print("test_adjacency_mat", np.array(test_adjacency_mat).shape)
print("entity2id", len(entity2id))
print("relation2id", len(relation2id))
print("headTailSelector", len(headTailSelector))
print("unique_entities_train", np.array(unique_entities_train).shape)

test_triples (5000, 3)
test_adjacency_mat (3, 5000)
entity2id 40943
relation2id 18
headTailSelector 18
unique_entities_train (40943,)


# Load batch size for training


In [ ]:
def load_data_main(args):
    train_data, validation_data, test_data, entity2id, relation2id, headTailSelector, unique_entities_train = build_data(
        args.data, is_unweigted=False, directed=True)

    if args.pretrained_emb:
        entity_embeddings, relation_embeddings = init_embeddings(os.path.join(args.data, 'entity2vec.txt'),
                                                                 os.path.join(args.data, 'relation2vec.txt'))
        print("Initialised relations and entities from TransE")

    else:
        entity_embeddings = np.random.randn(
            len(entity2id), args.embedding_size)
        relation_embeddings = np.random.randn(
            len(relation2id), args.embedding_size)
        print("Initialised relations and entities randomly")

    corpus = Corpus(args, train_data, validation_data, test_data, entity2id, relation2id, headTailSelector,
                    args.batch_size_gat, args.valid_invalid_ratio_gat, unique_entities_train, args.get_2hop)

    return corpus, torch.FloatTensor(entity_embeddings), torch.FloatTensor(relation_embeddings)

Corpus_, entity_embeddings, relation_embeddings = load_data_main(args)
torch.save(Corpus_, drive_folder + "Corpus_torch.pt")
torch.save(entity_embeddings, drive_folder + "entity_embeddings.pt")
torch.save(relation_embeddings, drive_folder + "relation_embeddings.pt")
node_neighbors_2hop = Corpus_.node_neighbors_2hop

number of unique_entities -> 40943
number of unique_entities -> 7802
number of unique_entities -> 7845
Initialised relations and entities from TransE
Graph created
length of graph keys is  40504


KeyboardInterrupt: ignored

# Loss

In [ ]:
def batch_gat_loss(gat_loss_func, train_indices, entity_embed, relation_embed):
    len_pos_triples = int(
        train_indices.shape[0] / (int(args.valid_invalid_ratio_gat) + 1))

    pos_triples = train_indices[:len_pos_triples]
    neg_triples = train_indices[len_pos_triples:]

    pos_triples = pos_triples.repeat(int(args.valid_invalid_ratio_gat), 1)

    source_embeds = entity_embed[pos_triples[:, 0]]
    relation_embeds = relation_embed[pos_triples[:, 1]]
    tail_embeds = entity_embed[pos_triples[:, 2]]

    x = source_embeds + relation_embeds - tail_embeds
    pos_norm = torch.norm(x, p=1, dim=1)

    source_embeds = entity_embed[neg_triples[:, 0]]
    relation_embeds = relation_embed[neg_triples[:, 1]]
    tail_embeds = entity_embed[neg_triples[:, 2]]

    x = source_embeds + relation_embeds - tail_embeds
    neg_norm = torch.norm(x, p=1, dim=1)

    y = -torch.ones(int(args.valid_invalid_ratio_gat) * len_pos_triples).cuda()

    loss = gat_loss_func(pos_norm, neg_norm, y)
    return loss

In [ ]:
# !cp /content/gdrive/My\ Drive/Corpus_torch.pt /content/data/WN18RR/

# Load data saved

In [ ]:
Corpus_ = torch.load(args.data + "Corpus_torch.pt")
entity_embeddings = torch.load(args.data + "entity_embeddings.pt")
relation_embeddings = torch.load(args.data + "relation_embeddings.pt")
node_neighbors_2hop = Corpus_.node_neighbors_2hop

# Define model

In [ ]:
print("Defining model")
model_gat = SpKBGATModified(entity_embeddings, relation_embeddings, args.entity_out_dim, args.entity_out_dim,
                            args.drop_GAT, args.alpha, args.nheads_GAT)
print("Only Conv model trained")
model_conv = SpKBGATConvOnly(entity_embeddings, relation_embeddings, args.entity_out_dim, args.entity_out_dim,
                              args.drop_GAT, args.drop_conv, args.alpha, args.alpha_conv,
                              args.nheads_GAT, args.out_channels)

if CUDA:
        model_conv.cuda()
        model_gat.cuda()

Defining model
Only Conv model trained


# Training KBGAT

In [ ]:
optimizer = torch.optim.Adam(
    model_gat.parameters(), lr=args.lr, weight_decay=args.weight_decay_gat)

scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer, step_size=500, gamma=0.5, last_epoch=-1)

gat_loss_func = nn.MarginRankingLoss(margin=args.margin)

current_batch_2hop_indices = torch.tensor([])
if(args.use_2hop):
    current_batch_2hop_indices = Corpus_.get_batch_nhop_neighbors_all(args, Corpus_.unique_entities_train, node_neighbors_2hop)

if CUDA:
    current_batch_2hop_indices = Variable(torch.LongTensor(current_batch_2hop_indices)).cuda()
else:
    current_batch_2hop_indices = Variable(torch.LongTensor(current_batch_2hop_indices))
    

epoch_losses = []   # losses of all epochs
print("Number of epochs {}".format(args.epochs_gat))

length of unique_entities  40559
Number of epochs 3600


In [ ]:
for epoch in range(args.epochs_gat):
    random.shuffle(Corpus_.train_triples)
    Corpus_.train_indices = np.array(
        list(Corpus_.train_triples)).astype(np.int32)

    model_gat.train()  # getting in training mode
    start_time = time.time()
    epoch_loss = []

    if len(Corpus_.train_indices) % args.batch_size_gat == 0:
        num_iters_per_epoch = len(
            Corpus_.train_indices) // args.batch_size_gat
    else:
        num_iters_per_epoch = (
            len(Corpus_.train_indices) // args.batch_size_gat) + 1

    for iters in range(num_iters_per_epoch):
        start_time_iter = time.time()
        train_indices, train_values = Corpus_.get_iteration_batch(iters)

        if CUDA:
            train_indices = Variable(
                torch.LongTensor(train_indices)).cuda()
            train_values = Variable(torch.FloatTensor(train_values)).cuda()

        else:
            train_indices = Variable(torch.LongTensor(train_indices))
            train_values = Variable(torch.FloatTensor(train_values))

        # forward pass
        entity_embed, relation_embed = model_gat(
            Corpus_, Corpus_.train_adj_matrix, train_indices, current_batch_2hop_indices)

        optimizer.zero_grad()

        loss = batch_gat_loss(
            gat_loss_func, train_indices, entity_embed, relation_embed)

        loss.backward()
        optimizer.step()

        epoch_loss.append(loss.data.item())

        end_time_iter = time.time()

    scheduler.step()
    epoch_losses.append(sum(epoch_loss) / len(epoch_loss))

    if (epoch > args.epochs_gat - 3):
        save_model(model_gat, args.data, epoch, args.output_folder)
        torch.save(epoch_losses, args.output_folder + "epoch_losses.pt")

Saving Model
Done saving Model
Saving Model
Done saving Model


In [ ]:
args.output_folder

'/content/output/WN18RR/'

### Load model GAT if not

In [ ]:
model_gat.load_state_dict(torch.load(
        '{}trained_{}.pt'.format("/content/output/WN18RR/", args.epochs_gat - 1)), strict=False)

<All keys matched successfully>

# Train Conv

In [ ]:
model_conv.final_entity_embeddings = model_gat.final_entity_embeddings
model_conv.final_relation_embeddings = model_gat.final_relation_embeddings

Corpus_.batch_size = args.batch_size_conv
Corpus_.invalid_valid_ratio = int(args.valid_invalid_ratio_conv)

optimizer = torch.optim.Adam(
model_conv.parameters(), lr=args.lr, weight_decay=args.weight_decay_conv)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=25, gamma=0.5, last_epoch=-1)

margin_loss = torch.nn.SoftMarginLoss()

epoch_losses = []   # losses of all epochs
print("Number of epochs {}".format(args.epochs_conv))

Number of epochs 200


In [ ]:
for epoch in range(args.epochs_conv):
    random.shuffle(Corpus_.train_triples)
    Corpus_.train_indices = np.array(
        list(Corpus_.train_triples)).astype(np.int32)

    model_conv.train()  # getting in training mode
    start_time = time.time()
    epoch_loss = []

    if len(Corpus_.train_indices) % args.batch_size_conv == 0:
        num_iters_per_epoch = len(
            Corpus_.train_indices) // args.batch_size_conv
    else:
        num_iters_per_epoch = (
            len(Corpus_.train_indices) // args.batch_size_conv) + 1

    for iters in range(num_iters_per_epoch):
        start_time_iter = time.time()
        train_indices, train_values = Corpus_.get_iteration_batch(iters)

        if CUDA:
            train_indices = Variable(
                torch.LongTensor(train_indices)).cuda()
            train_values = Variable(torch.FloatTensor(train_values)).cuda()

        else:
            train_indices = Variable(torch.LongTensor(train_indices))
            train_values = Variable(torch.FloatTensor(train_values))

        preds = model_conv(
            Corpus_, Corpus_.train_adj_matrix, train_indices)

        optimizer.zero_grad()

        loss = margin_loss(preds.view(-1), train_values.view(-1))

        loss.backward()
        optimizer.step()

        epoch_loss.append(loss.data.item())

        end_time_iter = time.time()

        # print("Iteration-> {0}  , Iteration_time-> {1:.4f} , Iteration_loss {2:.4f}".format(
        #     iters, end_time_iter - start_time_iter, loss.data.item()))

    scheduler.step()
    # print("Epoch {} , average loss {} , epoch_time {}".format(
    #     epoch, sum(epoch_loss) / len(epoch_loss), time.time() - start_time))
    epoch_losses.append(sum(epoch_loss) / len(epoch_loss))
    if (epoch > args.epochs_conv - 3):
        save_model(model_conv, args.data, epoch, args.output_folder)
        
torch.save(epoch_losses, args.output_folder + "epoch_losses_conv.pt")

In [ ]:
def evaluate_conv(args, unique_entities):
    model_conv = SpKBGATConvOnly(entity_embeddings, relation_embeddings, args.entity_out_dim, args.entity_out_dim,
                                 args.drop_GAT, args.drop_conv, args.alpha, args.alpha_conv,
                                 args.nheads_GAT, args.out_channels)
    

    model_conv.cuda()
    model_conv.eval()
    with torch.no_grad():
        Corpus_.get_validation_pred(args, model_conv, unique_entities)

In [ ]:
evaluate_conv(args, Corpus_.unique_entities_train)

In [ ]:
model_conv.load_state_dict(torch.load(
        '{0}trained_{1}.pth'.format(args.output_folder, args.epochs_conv - 1)), strict=False)
with torch.no_grad():
        Corpus_.get_validation_pred(args, model_conv, Corpus_.unique_entities_train)

Streaming output truncated to the last 5000 lines.
519
sample -  1 1
520
sample -  1 1
521
sample -  51 17
522
sample -  42 241
523
sample -  4635 955
524
sample -  1 1
525
sample -  7126 1209
526
sample -  5 1
527
sample -  10462 1059
528
sample -  1 1
529
529
sample -  1 1
530
sample -  790 189
531
sample -  18 23
532
sample -  334 48
533
sample -  6 9
534
sample -  2 1
535
sample -  102 3
536
sample -  20 4
537
sample -  8215 8109
538
sample -  1 1
539
sample -  26098 12428
540
sample -  1907 3207
541
sample -  3 4
542
sample -  1 3
543
sample -  1168 65
544
sample -  57 42
545
sample -  26204 14507
546
sample -  6 1
547
sample -  1151 3844
548
sample -  1 1
549
549
sample -  43 16
550
sample -  37 2
551
sample -  1388 250
552
sample -  1 2
553
sample -  33 134
554
sample -  1 1
555
sample -  1 1
556
sample -  1 1
557
sample -  1 1
558
sample -  4 6
559
sample -  2 3
560
sample -  1 1
561
sample -  28890 18432
562
sample -  1 4
563
sample -  1662 77
564
sample -  1 1
565
sample -  2

AttributeError: ignored

In [ ]:
output_folder

'/content/output/WN18RR/'